## 1. 라이브러리 설치

In [1]:
!pip install pdfplumber

## 2. Bedrock 설정 및 호출 함수 정의

In [2]:
import json
import boto3
from pprint import pprint

# Bedrock runtime 생성
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime"
)

In [3]:
def invoke_bedrock(
    query: str,
    model_id: str = "anthropic.claude-3-sonnet-20240229-v1:0",
    max_tokens: int = 4096,
    temperature: float = 1.0,
    top_p: float = 1.0,
    top_k: int = 250,
) -> str:

    # Request body 작성
    request_data = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": query},
                ],
            }
        ],
    }
    request_body = json.dumps(request_data)

    # 모델 호출
    response = bedrock_runtime.invoke_model(
        body=request_body,
        modelId=model_id,
        contentType="application/json",
        accept="application/json"
    )

    response_body = json.loads(response['body'].read())
    answer = response_body['content'][0]['text']
    return answer

## 3. PDF 문서 읽는 함수 정의¶

In [4]:
import pdfplumber

def get_texts_from_pdf(filepath: str, start_page:int, page_number:int):
    with pdfplumber.open(filepath) as pdf:
        number_of_pages = len(pdf.pages)
        end_page = start_page + page_number
        if end_page > number_of_pages:
            end_page = number_of_pages
            
        contents = ""
        for i in range(start_page, end_page):
            page = pdf.pages[i]
            contents += page.extract_text()
            contents += "\n\n"
        return contents

## 5. 챕터별로 요약하기

In [5]:
content = get_texts_from_pdf(filepath="./rag_data/Ferrous Markets Analytics Monthly_26 Jun 2024.pdf", start_page=0, page_number=99)
prompt = f"""

당신은 PDF 문서를 챕터별로 요약하는 AI 입니다.
주어진 <content> 태그 안에 있는 내용을 챕터 별로 요약해주세요.
각 챕터별로 전체적인 요약이 되어야 합니다.

<content>
{content}
</content>

<mandatory>
1. 숫자가 포함된 내용
</mandatory>

"""
result = invoke_bedrock(query=prompt, temperature=0, top_p=0)
print(result)

유럽 철강 요약:

- 독일 HRC 가격은 2024년 634유로/mt에서 2025년 720유로/mt로 상승할 것으로 예상됩니다.
- 독일 PMI 제조업지수는 2024년 41.9포인트에서 2025년 44.3포인트로 개선될 전망입니다. 
- 독일 철강 재고량은 2024년 1.2~1.3백만 mt 수준을 유지할 것으로 보입니다.

미국 철강 요약:

- 미국 HRC 가격은 2024년 783달러/st에서 2025년 930달러/st까지 상승할 것으로 예상됩니다.
- 미국 제조업 PMI는 2024년 48.7포인트에서 2025년 50.3포인트로 개선될 전망입니다.
- 미국 철강 가동률은 2024년 76.4%에서 2025년 79.5%까지 높아질 것으로 보입니다.
- 미국 철강 리드타임은 2024년 4.6주에서 2025년 5.2주로 길어질 전망입니다.  

중국 철강 요약:

- 2024년 중국 철강 소비량은 883백만 mt로 전년 대비 1.8% 감소할 것으로 예상됩니다.
- 부동산 부문 철강 수요는 247백만 mt로 8.2% 감소하겠지만, 인프라/기계/자동차 등 다른 부문 수요는 증가할 전망입니다.
